In [ ]:
# Importación de bibliotecas para análisis de métricas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import (classification_report, confusion_matrix, 
                           roc_curve, auc, roc_auc_score,
                           precision_recall_curve, average_precision_score)
from sklearn.preprocessing import label_binarize
from itertools import cycle
import warnings

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Bibliotecas para análisis de métricas importadas correctamente")
print("📊 Configuración de visualización establecida")


In [ ]:
# Cargar modelo y datos necesarios
try:
    # Cargar datos preprocesados
    processed_datasets = joblib.load('../models/processed_datasets.joblib')
    label_encoder = joblib.load('../models/label_encoder.joblib')
    test_results = joblib.load('../models/test_results.joblib')
    
    # Obtener el nombre del mejor modelo
    best_model_name = test_results['model_name']
    model_filename = f'../models/best_model_{best_model_name.replace(" ", "_")}.joblib'
    best_model = joblib.load(model_filename)
    
    print("✅ Modelo y datos cargados correctamente")
    print(f"🏆 Mejor modelo: {best_model_name}")
    
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("📋 Asegúrate de ejecutar primero los notebooks de preprocesamiento y benchmarking")

# Obtener datos de prueba del dataset principal
main_dataset = processed_datasets['StandardScaler_OneHot']
X_test = main_dataset['X_test']
y_test = main_dataset['y_test']

# Hacer predicciones
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

print(f"\n📊 INFORMACIÓN DEL CONJUNTO DE PRUEBA:")
print(f"   🧪 Tamaño: {X_test.shape[0]} muestras")
print(f"   📋 Features: {X_test.shape[1]}")
print(f"   🎯 Clases: {len(label_encoder.classes_)}")

print(f"\n🔮 PREDICCIONES GENERADAS:")
print(f"   ✅ Predicciones discretas: {y_pred.shape}")
print(f"   📊 Probabilidades: {y_pred_proba.shape}")

# Mostrar distribución de predicciones
pred_distribution = pd.Series(y_pred).value_counts().sort_index()
print(f"\n📊 DISTRIBUCIÓN DE PREDICCIONES:")
for class_idx, count in pred_distribution.items():
    class_name = label_encoder.inverse_transform([class_idx])[0]
    percentage = (count / len(y_pred)) * 100
    print(f"   {class_name}: {count} ({percentage:.1f}%)")


In [ ]:
# Generar reporte de clasificación detallado
class_names = label_encoder.classes_

print("📋 REPORTE DE CLASIFICACIÓN DETALLADO")
print("="*60)

# Reporte de clasificación con nombres de clases
classification_rep = classification_report(
    y_test, y_pred, 
    target_names=class_names,
    output_dict=True
)

# Mostrar reporte formateado
print(classification_report(y_test, y_pred, target_names=class_names))

# Crear DataFrame para mejor análisis
report_df = pd.DataFrame(classification_rep).transpose()
report_df = report_df.round(4)

print("\n📊 MÉTRICAS POR CLASE:")
print("="*50)
for class_name in class_names:
    if class_name in report_df.index:
        precision = report_df.loc[class_name, 'precision']
        recall = report_df.loc[class_name, 'recall']
        f1 = report_df.loc[class_name, 'f1-score']
        support = int(report_df.loc[class_name, 'support'])
        
        print(f"\n🏷️ Clase: {class_name}")
        print(f"   📈 Precision: {precision:.4f}")
        print(f"   📈 Recall: {recall:.4f}")
        print(f"   📈 F1-Score: {f1:.4f}")
        print(f"   📊 Support: {support} muestras")

# Métricas globales
macro_avg = report_df.loc['macro avg']
weighted_avg = report_df.loc['weighted avg']

print(f"\n🌐 MÉTRICAS GLOBALES:")
print(f"   📊 Macro Average:")
print(f"      • Precision: {macro_avg['precision']:.4f}")
print(f"      • Recall: {macro_avg['recall']:.4f}")
print(f"      • F1-Score: {macro_avg['f1-score']:.4f}")
print(f"   📊 Weighted Average:")
print(f"      • Precision: {weighted_avg['precision']:.4f}")
print(f"      • Recall: {weighted_avg['recall']:.4f}")
print(f"      • F1-Score: {weighted_avg['f1-score']:.4f}")

# Guardar reporte de clasificación
report_df.to_csv('../reports/classification_report.csv')
print(f"\n💾 Reporte guardado: ../reports/classification_report.csv")

# Guardar reporte de clasificación en texto
with open('../reports/classification_report.txt', 'w') as f:
    f.write(f"Reporte de Clasificación - Modelo: {best_model_name}\n")
    f.write("="*60 + "\n\n")
    f.write(classification_report(y_test, y_pred, target_names=class_names))

print(f"💾 Reporte en texto guardado: ../reports/classification_report.txt")


In [ ]:
# Crear y visualizar la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Crear visualización de la matriz de confusión
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'📊 MATRIZ DE CONFUSIÓN - {best_model_name}\n🎯 Análisis de Errores de Clasificación', 
             fontsize=16, fontweight='bold', y=1.02)

# Matriz de confusión en números absolutos
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0], cbar_kws={'shrink': 0.8})
axes[0].set_title('📈 Valores Absolutos', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Valor Real')

# Matriz de confusión normalizada (porcentajes)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Oranges',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[1], cbar_kws={'shrink': 0.8})
axes[1].set_title('📊 Porcentajes (Normalizado)', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Predicción')
axes[1].set_ylabel('Valor Real')

plt.tight_layout()
plt.savefig('../reports/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Análisis detallado de la matriz de confusión
print("📊 ANÁLISIS DETALLADO DE LA MATRIZ DE CONFUSIÓN")
print("="*55)

total_samples = cm.sum()
correct_predictions = np.trace(cm)
accuracy = correct_predictions / total_samples

print(f"🎯 ESTADÍSTICAS GENERALES:")
print(f"   📊 Total de muestras: {total_samples}")
print(f"   ✅ Predicciones correctas: {correct_predictions}")
print(f"   ❌ Predicciones incorrectas: {total_samples - correct_predictions}")
print(f"   📈 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

print(f"\n🔍 ANÁLISIS POR CLASE:")
for i, class_name in enumerate(class_names):
    # Verdaderos positivos, falsos positivos, falsos negativos
    tp = cm[i, i]
    fp = cm[:, i].sum() - tp
    fn = cm[i, :].sum() - tp
    tn = total_samples - tp - fp - fn
    
    # Métricas por clase
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    print(f"\n🏷️ Clase: {class_name}")
    print(f"   ✅ Verdaderos Positivos (TP): {tp}")
    print(f"   ❌ Falsos Positivos (FP): {fp}")
    print(f"   ❌ Falsos Negativos (FN): {fn}")
    print(f"   ✅ Verdaderos Negativos (TN): {tn}")
    print(f"   📈 Precision: {precision:.4f}")
    print(f"   📈 Recall (Sensibilidad): {recall:.4f}")
    print(f"   📈 Especificidad: {specificity:.4f}")

# Identificar errores más comunes
print(f"\n❌ ERRORES MÁS COMUNES:")
print("="*30)

# Encontrar las confusiones más frecuentes (excluyendo la diagonal)
errors = []
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i, j] > 0:
            errors.append((class_names[i], class_names[j], cm[i, j]))

# Ordenar por frecuencia de error
errors.sort(key=lambda x: x[2], reverse=True)

for true_class, pred_class, count in errors[:5]:  # Top 5 errores
    percentage = (count / total_samples) * 100
    print(f"   🔄 {true_class} → {pred_class}: {count} veces ({percentage:.2f}%)")

# Guardar matriz de confusión
np.savetxt('../reports/confusion_matrix.csv', cm, delimiter=',', fmt='%d')
print(f"\n💾 Matriz de confusión guardada: ../reports/confusion_matrix.csv")


In [ ]:
# Calcular curvas ROC y AUC para clasificación multiclase
n_classes = len(class_names)

# Binarizar las etiquetas para ROC multiclase
y_test_binarized = label_binarize(y_test, classes=range(n_classes))

# Si solo hay 2 clases, label_binarize devuelve un array 1D
if n_classes == 2:
    y_test_binarized = np.column_stack([1 - y_test_binarized, y_test_binarized])

print("📈 ANÁLISIS DE CURVAS ROC Y AUC")
print("="*40)

# Calcular ROC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calcular micro-average ROC
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Calcular macro-average ROC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Visualizar curvas ROC
plt.figure(figsize=(14, 10))

# Colores para cada clase
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'purple'])

# Plotear ROC para cada clase
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC {class_names[i]} (AUC = {roc_auc[i]:.3f})')

# Plotear micro y macro average
plt.plot(fpr["micro"], tpr["micro"],
         label=f'Micro-average ROC (AUC = {roc_auc["micro"]:.3f})',
         color='deeppink', linestyle=':', linewidth=3)

plt.plot(fpr["macro"], tpr["macro"],
         label=f'Macro-average ROC (AUC = {roc_auc["macro"]:.3f})',
         color='navy', linestyle=':', linewidth=3)

# Línea de referencia (clasificador aleatorio)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Clasificador Aleatorio (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)', fontsize=12, fontweight='bold')
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad)', fontsize=12, fontweight='bold')
plt.title(f'📈 CURVAS ROC MULTICLASE - {best_model_name}\\n🎯 Receiver Operating Characteristic', 
          fontsize=16, fontweight='bold', pad=20)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)

# Añadir anotaciones
plt.text(0.6, 0.2, f'Mejor Rendimiento:\\nMacro AUC = {roc_auc["macro"]:.3f}\\nMicro AUC = {roc_auc["micro"]:.3f}',
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8),
         fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# Mostrar AUC por clase
print(f"📊 AUC POR CLASE:")
for i, class_name in enumerate(class_names):
    print(f"   🏷️ {class_name}: {roc_auc[i]:.4f}")

print(f"\n📊 AUC PROMEDIO:")
print(f"   🔍 Micro-average: {roc_auc['micro']:.4f}")
print(f"   🔍 Macro-average: {roc_auc['macro']:.4f}")

# Interpretación de AUC
macro_auc = roc_auc['macro']
if macro_auc >= 0.9:
    interpretation = "Excelente"
elif macro_auc >= 0.8:
    interpretation = "Bueno"
elif macro_auc >= 0.7:
    interpretation = "Aceptable"
elif macro_auc >= 0.6:
    interpretation = "Pobre"
else:
    interpretation = "Muy Pobre"

print(f"\n💡 INTERPRETACIÓN DEL RENDIMIENTO:")
print(f"   📈 AUC Macro: {macro_auc:.4f} → {interpretation}")

# Guardar resultados ROC
roc_results = {
    'class_auc': {class_names[i]: roc_auc[i] for i in range(n_classes)},
    'micro_auc': roc_auc['micro'],
    'macro_auc': roc_auc['macro']
}

import json
with open('../reports/roc_auc_results.json', 'w') as f:
    json.dump(roc_results, f, indent=2)

print(f"\n💾 Resultados ROC guardados: ../reports/roc_auc_results.json")


In [ ]:
# Generar resumen final completo
print("🎯 RESUMEN FINAL DEL ANÁLISIS DE MÉTRICAS")
print("="*50)

print(f"\n🏆 MODELO SELECCIONADO: {best_model_name}")
print(f"📊 Dataset: Clasificación de categorías de venta (retail)")
print(f"🎯 Problema: Clasificación multiclase ({n_classes} clases)")

print(f"\n📈 RENDIMIENTO EN CONJUNTO DE PRUEBA:")
print(f"   🎯 Accuracy: {test_results['test_accuracy']:.4f}")
print(f"   📊 Precision (macro): {test_results['test_precision']:.4f}")
print(f"   📊 Recall (macro): {test_results['test_recall']:.4f}")
print(f"   📊 F1-Score (macro): {test_results['test_f1']:.4f}")
print(f"   📈 AUC (macro): {roc_auc['macro']:.4f}")

print(f"\n🔍 ANÁLISIS POR CLASE:")
for i, class_name in enumerate(class_names):
    if class_name in report_df.index:
        precision = report_df.loc[class_name, 'precision']
        recall = report_df.loc[class_name, 'recall']
        f1 = report_df.loc[class_name, 'f1-score']
        auc_score = roc_auc[i]
        support = int(report_df.loc[class_name, 'support'])
        
        print(f"   🏷️ {class_name}:")
        print(f"      • Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f} | AUC: {auc_score:.3f}")
        print(f"      • Muestras: {support}")

print(f"\n💡 INSIGHTS CLAVE:")
# Identificar la mejor y peor clase
best_class_idx = np.argmax([roc_auc[i] for i in range(n_classes)])
worst_class_idx = np.argmin([roc_auc[i] for i in range(n_classes)])

best_class = class_names[best_class_idx]
worst_class = class_names[worst_class_idx]

print(f"   🏆 Mejor clase predicha: {best_class} (AUC: {roc_auc[best_class_idx]:.3f})")
print(f"   📉 Clase más difícil: {worst_class} (AUC: {roc_auc[worst_class_idx]:.3f})")

# Análisis de balance del dataset
class_distribution = pd.Series(y_test).value_counts().sort_index()
most_frequent_class = label_encoder.inverse_transform([class_distribution.idxmax()])[0]
least_frequent_class = label_encoder.inverse_transform([class_distribution.idxmin()])[0]

print(f"   📊 Clase más frecuente: {most_frequent_class} ({class_distribution.max()} muestras)")
print(f"   📊 Clase menos frecuente: {least_frequent_class} ({class_distribution.min()} muestras)")

# Evaluar si hay desbalance
max_count = class_distribution.max()
min_count = class_distribution.min()
imbalance_ratio = max_count / min_count

if imbalance_ratio > 2:
    print(f"   ⚠️ Dataset desbalanceado (ratio: {imbalance_ratio:.1f}:1)")
else:
    print(f"   ✅ Dataset relativamente balanceado (ratio: {imbalance_ratio:.1f}:1)")

print(f"\n📋 ARCHIVOS GENERADOS:")
print(f"   📄 ../reports/classification_report.csv")
print(f"   📄 ../reports/classification_report.txt")
print(f"   📊 ../reports/confusion_matrix.png")
print(f"   📊 ../reports/confusion_matrix.csv")
print(f"   📈 ../reports/roc_curve.png")
print(f"   📄 ../reports/roc_auc_results.json")

print(f"\n🎯 RECOMENDACIONES:")
if test_results['test_f1'] >= 0.8:
    print("   ✅ Modelo con excelente rendimiento, listo para producción")
elif test_results['test_f1'] >= 0.7:
    print("   ✅ Modelo con buen rendimiento, considerar mejoras adicionales")
elif test_results['test_f1'] >= 0.6:
    print("   ⚠️ Modelo con rendimiento aceptable, recomendado optimizar")
else:
    print("   ❌ Modelo con rendimiento pobre, requiere mejoras significativas")

if imbalance_ratio > 2:
    print("   📊 Considerar técnicas de balanceo de clases (SMOTE, undersampling)")

if worst_class_idx != best_class_idx:
    print(f"   🎯 Enfocar mejoras en la predicción de la clase '{worst_class}'")

print(f"\n✅ ANÁLISIS DE MÉTRICAS COMPLETADO")
print(f"🎯 El modelo {best_model_name} está listo para implementación")
